In [ ]:
# requirements
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

# Загрузка документов
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter
)

# Эмбеддинги
from chromadb.config import Settings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
# from langchain.embeddings import LlamaCppEmbeddings

# QnA цепочка
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

import torch
torch.cuda.is_available()

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# model_path = "./data/weights/mixtral-8x7b-instruct-v0.1.Q6_K.gguf"
model_path = "./data/weights/openchat_3.5.Q5_K_M.gguf"
# model_path = "./data/weights/Mistral-7B-Instruct-v0.3.Q6_K.gguf"

embed_model_path = "./data/weights/intfloat_multilingual-e5-large"
embed_model_path_kwargs = {"device": "cuda:0"}
index_path = "./data/index"

data_file = "./data/data/bzd.docx"

In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path, 
    temperature=0.2,
    max_new_tokens=10000,
    context_window=16379-1000,
    generate_kwargs={},
    # n_ctx=8192,
    n_gpu_layers=50, 
    # n_threads=6, 
    # n_batch=521, 
    verbose=True,
    callback_manager=callback_manager
)

In [ ]:
res = llm.invoke(
      "Q: Кто из знаменитостей родился в год распада СССР? A: ", # Prompt
      stop=["Q:", "\n"], 
      echo=True,
) 
print(res)

In [ ]:
loader = Docx2txtLoader(data_file)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=embed_model_path, model_kwargs=embed_model_path_kwargs)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=True),
)

retriever = db.as_retriever(k=10)

In [ ]:
question = "Устойчивость работы хозяйственного объекта определяется по"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain.invoke({"query": question}))

In [ ]:
question = "Аварией считается утечка нефти в щбьеме"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain.invoke({"query": question}))

In [ ]:
question = "Пожарная техника в зависимости от способа пожаротушения подразделяется на"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain.invoke({"query": question}))

In [ ]:
question = "Сколько классов опасных производственных объектов существует"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())
print(qa_chain.invoke({"query": question}))

In [ ]:
question = "Кто такой джастин бибер?"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain.invoke({ "query": question}))

In [ ]:
sys_templ = '''Ответь на вопрос пользователя на русском языке. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, скажи "Я не знаю".
После основного ответа напиши степень уверенности в ответе. 
<context>
{context}
</context>
'''
qa_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            sys_templ,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
combine_docs_chain = create_stuff_documents_chain(llm, qa_prompt)

# question = "Кто такой джастин бибер?"
question = "Аварией считается утечка нефти в обьеме"
# question = "Устойчивость работы хозяйственного объекта определяется по"

# Поиск по базе данных
docs = retriever.invoke(question)

result = combine_docs_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content=question)
        ],
    }
)

print(result)

In [ ]:
# question = "Кто такой джастин бибер?"
question = "Аварией считается утечка нефти в щбьеме"
# question = "Устойчивость работы хозяйственного объекта определяется по"

template = '''Ответь на вопрос пользователя на русском языке. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, скажи "Я не знаю".
После основного ответа напиши степень уверенности в ответе. 

Context: {context}

Human: {question}
Assistant:
'''
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
prompt = PromptTemplate(input_variables=["context",  "question"], template=template)
    
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, memory=memory, chain_type_kwargs={'prompt': prompt})
print(qa_chain({"query": question}))